Let's try to solve a first order differential equation using a guess (model) function. This function will be used to adjust the parameters so that the left- and right-hand side of the ODE are as close as possible. 

In [1]:
import os

os.chdir('Optimizers/Gradient-Descent/python/')
import GD as GD 
os.chdir('../../../')

import numpy as np

For example, consider the ODE
$$
\dfrac{dy}{dt}= G(y(t);t) \;,
$$
for $t \in [0,T]$ with $y(0)=y_0$.

we can use a function, $f(t;\{w\})$, that depends on severan parameters (e.g. $f(t;\{w\}) = \sum_{i=0}^{N} w_i \ t^i$) in order to model the solution.

The key point here is that we are not going to fit the model in the entire $t \in [0,T]$. we are only going to ask for a the values of $y$ for a few points in $t$. 

Given a list of $t=t_1,t_2 \dots$ for which we wish to find $y(t_i)$. We, then, minimize 
$$
Q(\{w\}) = \dfrac{1}{2}\Bigg[\left(f(0;\{w\})-y_0\right)^2 +\left(\dfrac{df(t;\{w\})}{dt}- G\Big( \ f(t;\{w\}),t\Big)\right)_{t=t_1}^2 \Bigg]\;.
$$
When this is minimized, we have the values for the $w$ which define a function that has the same initial condition as $y$ and obeys the ODE at $t=t_1$. Once this is done $t=t_1$, we store the value $y(t_1)=f(t_1;\{w\})$, and we do the same for the other points.

What we need is $\nabla_{w_i} \ Q(\{w\})$, which will be used from the optimizer.

Let's say we want to solve 
$$
\dfrac{dy}{dt}=-sin(t) \;, \; \text{ with} \;\; y(0)=1 \;.
$$

In order to solve it, we will use a model function:
$$
y=w_0 \ sin(w_1 \ t) + w_2 \ cos(w_3 \ t).
$$

Define the model function.

In [2]:
class modelFunc(GD.modelBase):
    def __init__(self,dimensions,w0,h=1e-8):
        GD.modelBase.__init__(self,dimensions,w0)
        
        self.dsdx=[0 for _ in range(dimensions[1])]
        self.h=h
        
    def __call__(self):
        self.signal[0]=np.sin(self.input*self.w[1])*self.w[0]+np.cos(self.input*self.w[3])*self.w[2]
        
        
    def derivative_x(self):
        _x=self.input
        heff=np.abs(_x)*self.h+self.h
        
        for j in range(self.dimensions[1]):
            _x-=heff
            self.setInput(_x)
            self()
            f0=self.signal[j]
            
            _x+=2*heff
            self.setInput(_x)
            self()
            f1=self.signal[j]
            
            self.dsdx[j]=(f1-f0)/(2*heff)
            
            _x-=heff
            self.setInput(_x)
            
    
    def derivative_w(self,i):
        heff=np.abs(self.w[i])*self.h+self.h
        
        for j in range(self.dimensions[1]):
            self.w[i]-=heff
            self()
            f0=self.signal[j]
            
            self.w[i]+=2*heff
            self()
            f1=self.signal[j]
            
            self.dsdw[j]=(f1-f0)/(2*heff)
            
            self.w[i]-=heff


Define the region in which you want to solve the ODE, and the initial condition.

In [3]:
class Boundary:
    def __init__(self,model,points,y0):
        '''
        model: the model function
        points: the points at which we want the solution
        y0: the value of y at points[0]
        '''
        self.model=model
        self.points=points[:]
        self.y0=y0
        
        self.currentPoint=0#the current index of points we are evaluating
        self.maxPoints=len(points)
        self.complete=False#switch to True when self.currentPoint=self.maxPoints
        
        
        self.boundary_diff=[0 for _ in range(model.dimensions[1])]
        
        self.model.setInput(self.points[0])#start at points[0]
        
    def boundaryDifference(self):
        '''
        get the difference between self.y0 and the output of the model for model.input=points[0]
        '''
        tmp_x=self.model.input
        
        self.model.setInput(self.points[0])
        
        self.model()
        for i,_y in enumerate(self.y0):
            self.boundary_diff[i]=self.model.signal[i] -_y 
            
        self.model.setInput(tmp_x)
        
            
        
            
    def nextPoint(self):
        '''get the next point in points and set it ass input for the model'''
        if self.currentPoint==self.maxPoints-1:
            self.complete=True
        else:
            self.currentPoint+=1
            self.model.setInput(self.points[self.currentPoint])


In [4]:
model=modelFunc([1,1],[0.1,0.3,0.4,0.5])

In [5]:
boundary=Boundary(model,np.linspace(0,1,5),[1])

define the ODE, which also defines the loss.

In [6]:
class DifferentialEquation:
    def __init__(self,Model,Boundary):
        
        self.model=Model
        self.boundary=Boundary
        
        self.DE=[0 for _ in range(self.model.dimensions[1])]#this should be LHS-RHS, which we want to be 0 (see definition of __call__)
        
        self.grad=[0 for i in  range(self.model.dim)]
        
        
    def __call__(self):
        self.model.derivative_x()
        for i,dMdt in enumerate(self.model.dsdx):
            self.DE[i] = dMdt+np.sin(self.model.input)
        
        
    
    def averageLoss(self):
        self.boundary.boundaryDifference()
        self()
        avrgLoss=0
        for i in range(self.model.dimensions[1]):
            avrgLoss+=(self.boundary.boundary_diff[i]**2+self.DE[i]**2)/2.
        
        return avrgLoss/float(self.model.dimensions[1])
        

    def averageGrad(self,h=1e-5):
        '''Get the gradient of the averge loss at current point and a random boundary point'''
        
        for dim in range(self.model.dim):
            heff=h*np.abs(self.model.w[dim])+h
            
            self.model.w[dim]-=heff
            Q0=self.averageLoss()

            self.model.w[dim]+=2*heff
            Q1=self.averageLoss()
            
            self.grad[dim]=(Q1-Q0)/(2.*heff)

            self.model.w[dim]-=heff
        


In [7]:
model=modelFunc([1,1],[0,0,.3,2])
boundary=Boundary(model,np.linspace(0,3.14159,10),[-1])

In [8]:
ODE=DifferentialEquation(model,boundary)
# gd=GD.VanillaGD(ODE,alpha=1e-2)
# gd=GD.RMSpropGD(ODE,gamma=1-1e-2,epsilon=1e-5,alpha=1e-2)
# gd=GD.AdaDeltaGD(ODE,gamma=0.99,epsilon=1e-5,alpha=1)
# gd=GD.AdamGD(Q,beta_m=0.9,beta_v=0.999,epsilon=1e-8,alpha=1e-2)
# gd=GD.AdaMaxGD(ODE,beta_m=0.9,beta_v=0.999,epsilon=1e-8,alpha=1e-1)
gd=GD.NAdamGD(ODE,beta_m=0.9,beta_v=0.999,epsilon=1e-8,alpha=1e-2)

In [9]:
t=[]
y=[]
while not boundary.complete:
    gd.run(abs_tol=1e-3, rel_tol=1e-3, step_break=5000,max_step=5000)

    model()
    
    t.append(model.input)
    y.append(model.signal[0])
    model.w=[0,0,.3,2]
    
    boundary.nextPoint()
    
t=np.array(t)
y=np.array(y)

In [14]:
-np.cos(t)

array([-1.        , -0.93969272, -0.76604482, -0.50000077, -0.17364934,
        0.17364673,  0.49999847,  0.76604312,  0.93969181,  1.        ])

In [11]:
y

array([-1.        , -1.24762572, -2.35243963, -0.91343384, -3.10716878,
        0.34611009,  0.85887029,  0.89945666, -1.16272495, -0.99992536])

In [12]:
model.dsdx

[-0.007943209743849365]

In [13]:
model.w

[0, 0, 0.3, 2]